#Abstract
This work was focused on solving a Stiff ODE. In this work it was shown that ordinary ways (which were examined in other assignments) of solving ODEs do not work when it comes to a Stiff ODE, since the inaccuracy in calculations increases drastically if the step size is not small enougth. According to this work, Implicit Euler's method works without significant degradation in accuracy even when it comes to solving Stiff ODE.

#Introduction
The main task of this work was to solve a Stiff ODE.
The ODE which was to be solved looks as following:
$$y'= f(t,y)= - 1000(y-\sin t) + \cos t ,$$
The initial condition was $ y(0)=1$. Here $1000$ could be replaced with a constant $a$. After replacing it, got:
$$y'= f(t,y)= - a(y-\sin t) + \cos t , \quad y(0) = 1$$
In order to find the solution I first solved the equation analytically, then I tried to obtain the solution using explicit Euler's method. Then, using RMS metric I compared the obtained solution with the analytical one. My hypothesis was that the answer would be far from the ideal. Then I used the implicit method for solving the given Stiff ODE.

In [ ]:
import numpy as np
def rms(analytical, experimental):
  return np.sqrt(np.mean(np.power(analytical-experimental, 2)))
"""
CONSTANT
"""
a = 1000
interval = np.linspace(0, 1, 1000)
epsilon = 1e-3

### Obtaining the analytical solution
The given ODE can be solved using Integrating Factor. <br> 
The solution is: $y = e^{-at} + sin(t)$ <br>


In [ ]:
def y_analytical(t):
  return np.exp(-a * t) + np.sin(t)
true_solution = y_analytical(interval)

In [ ]:
"""
From the initial condition we have
"""
y_initial = 1 

### Explicit method 
This method states that if we define $f(t, y(t)) = y'(t)$ such that the initial condition ($y'(t_initial) = y_initial$) is kept. We can find the next value of y by the following formula:
$$y_{n+1} = y_n + h * f(t_n, y_n)$$
Where h is stepsize. Here, since we are dealing with a Stiff ODE, h should be chose carefully. In order not to use the value which is too big, I will use 1e-3

In [ ]:
h = 0.1

In [ ]:
class ExplicitSolver():
  def __init__(self, t_init, t_final, y_initial, step_size):
    self.y = []
    self.t_init = t_init
    self.t_final = t_final
    self.h = step_size
    self.t = np.linspace(t_init, t_final, int( (t_final - t_init) / self.h))
    self.y.append(y_initial)

  def f(self, t, y):
    return -a * (y - np.sin(t)) + np.cos(t)

  def solve(self):
    for i in range(1, self.t.shape[0]):
      self.y.append(self.y[-1] + self.h * self.f(self.t[i-1], self.y[-1]))
    return np.array(self.y)

  def search(self):
    pred = self.solve()
    if rms(y_analytical(self.t), pred) >= epsilon:
      self.h /= 2.0
      self.t = np.linspace(self.t_init, self.t_final, int( (self.t_final - self.t_init) / self.h))
      self.y = [self.y[0]]
      return self.search()
    else:
      return self.y, self.h, rms(y_analytical(self.t), pred)


In [ ]:
t = np.linspace(0, 1, 1000)
em = ExplicitSolver(0, 1, y_initial, h)

In [ ]:
em_solution, best_h, best_rms = em.search()


In [ ]:
best_h, best_rms

(9.765625e-05, 0.0007936540355475494)

So the critical value of h is 9.765625e-05. It has RMS of 0.0007936540355475494 <br>
However, it requires the function to be calculated more than 10K times.

#Implicit method
Before talking about this method it is needed to mention why the explicit method failed. The stepsize for the explicit method shoold be too small (9.765e-5) in order to satisfy stability condition. To solve this problem, we can change the derivation of the $y_{n+1}$ including in it the stability condition. Then $$x_{n+1} = \frac{x_n}{1 + h*k}$$
Substituting our function, we get: $$y_{n+1} = y_n + h * f(t, y) = y_n + h * (-a * (y_n - sin(t)) + cos(t)) = \frac{y_n + h*(asin(t) + cos(t))}{1 + a*h}$$ 

In [ ]:
class ImplicitSolver():
  def __init__(self, t_init, t_final, y_initial, step_size):
    self.y = []
    self.t_init = t_init
    self.t_final = t_final
    self.h = step_size
    self.t = np.linspace(t_init, t_final, int( (t_final - t_init) / self.h))
    self.y.append(y_initial)

  def f(self, t, y):
    return -a * (-np.sin(t)) + np.cos(t)

  def solve(self):
    for i in range(1, self.t.shape[0]):
      self.y.append(self.y[-1] + self.h * self.f(self.t[i-1] + self.h, self.y[-1]))
      self.y[-1] = self.y[-1] / (1 + a*self.h)
    return np.array(self.y)

  def search(self):
    pred = self.solve()
    if rms(y_analytical(self.t), pred) >= epsilon:
      self.h /= 2.0
      self.t = np.linspace(self.t_init, self.t_final, int( (self.t_final - self.t_init) / self.h))
      self.y = [self.y[0]]
      return self.search()
    else:
      return self.y, self.h, rms(y_analytical(self.t), pred)

In [ ]:
im = ImplicitSolver(0, 1, 1, 0.5)

In [ ]:
im_values, bset_h, best_rms = im.search()

In [ ]:
bset_h

0.0001220703125

Now we have started with even a higher value of h, but still were able to find a stable solution. It required only 8K iterations.

#Conclusion
From the above work it is clear that implicit Euler's method goves result of the same accuracy as the Explicit method, but it requires significantly less steps, since it can operate with a big step size (while the Explicit method requires the step size to be small enough no to be unstable). 